In [ ]:
!git clone https://github.com/jaimeh94/Point-Dose-Calculation_TG-43.git

Cloning into 'Point-Dose-Calculation_TG-43'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 2), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [1]:
!pip install pydicom

In [1]:
import pydicom
import numpy as np
import pandas as pd


In [2]:
# %cd ..
RT_Plan = pydicom.dcmread('./Martha Altamar BQT/RP.1.3.6.1.4.1.2452.6.1116510183.1322412770.3776939407.874610829.dcm')

In [3]:
total_time = RT_Plan.ApplicationSetupSequence[0].ChannelSequence[0].ChannelTotalTime
total_time_weight = RT_Plan.ApplicationSetupSequence[0].ChannelSequence[0].FinalCumulativeTimeWeight

In [4]:
Catheter = RT_Plan.ApplicationSetupSequence[0].ChannelSequence[0].BrachyControlPointSequence
cum_time=0
time_dwell=0
Position=[]
for i in range(len(Catheter)):
    if i%2==1:
        raw_time_dwell = Catheter[i].CumulativeTimeWeight - cum_time
        # print(Catheter[i].ControlPoint3DPosition)
        dwell = Catheter[i].ControlPoint3DPosition
        # print(Catheter[i].CumulativeTimeWeight)
        cum_time = Catheter[i].CumulativeTimeWeight
        time_dwell=round(10*raw_time_dwell*total_time/total_time_weight,2)
        
        dwell.append(time_dwell)
        Position.append(np.array(dwell)/10)
        print(dwell)
        print(time_dwell)
        # print(time_dwell*total_time/total_time_weight)
        print()

[-21.877565, 4.990215, -14.851535, 2969.55]
2969.55

[-23.614176, 3.079813, -24.512516, 1098.76]
1098.76

[-25.323645, 1.182064, -34.179772, 445.44]
445.44

[-26.440784, -0.439579, -43.983968, 1098.76]
1098.76

[-27.557924, -2.061223, -53.788165, 2969.61]
2969.61



In [5]:
# RT_Plan.ApplicationSetupSequence[0].ChannelSequence[0].BrachyControlPointSequence[1].ControlPoint3DPosition
# s=Catheter[i].ControlPoint3DPosition
# s.append(3)
# np.array(s)
Position = pd.DataFrame(Position,columns=['x','y','z','time'])

In [6]:
points = RT_Plan.DoseReferenceSequence
for i in range(len(points)):
  print(points[i].DoseReferenceDescription)
  print(points[i].DoseReferencePointCoordinates)
  print(points[i].TargetPrescriptionDose)
# RT_Plan.DoseReferenceSequence[0]

V1
[-30.498478, -36.154693, -9.760086]
2.083688
V-ICRU
[-30.630797, -25.840958, -19.776421]
3.829628
V2
[-29.773411, -35.703611, -28.741108]
2.694996
A2
[-45.410141, 3.061159, -21.976970]
6.134374
A3
[-0.778892, 3.218761, -27.723862]
5.848383
A4
[-47.910672, -0.517739, -41.494625]
6.283706
A5
[-3.279423, -0.360138, -47.241517]
5.733538
R1
[-25.032631, 36.028478, -22.230613]
3.255357
R2
[-28.285661, 32.072904, -48.386848]
3.367051


In [7]:
type(points[3].DoseReferencePointCoordinates)

pydicom.multival.MultiValue

In [8]:
# import numpy as np
calc_point = np.array(points[3].DoseReferencePointCoordinates)/10
calc_point

array([-4.5410141,  0.3061159, -2.197697 ])

In [14]:
%cd Point-Dose-Calculation_TG-43/

/content/Point-Dose-Calculation_TG-43


In [9]:
from Iridium192 import Ir_192
from datetime import datetime
fuente = Ir_192(CalDate=datetime(2021,4,22,5,22), RAKR=52570)

In [10]:
rawPlanDate = RT_Plan.SourceSequence[0].SourceStrengthReferenceDate
rawPlanTime = RT_Plan.SourceSequence[0].SourceStrengthReferenceTime

In [11]:
PlanDate = datetime(int(rawPlanDate[:4]),int(rawPlanDate[4:6]),int(rawPlanDate[6:8]),int(rawPlanTime[:2]),int(rawPlanTime[2:4]))

In [12]:
from DoseRate import Dose_Rate
Dose_Rate(Position,calc_point,fuente,PlanDate)

ValueError: Unable to coerce to Series, length must be 4: given 3

In [34]:
def Dose_Rate(Position[['x','y','z']],calc_point,fuente,PlanDate):
    
    deltaPlanCal = (PlanDate-fuente.CalDate).days + (PlanDate-fuente.CalDate).seconds/(24*3600)

    Sk = fuente.RAKR*np.exp(-np.log(2)*deltaPlanCal/fuente.MeanLife)
    r = calc_point-Position
    a = pd.DataFrame([Position.loc[i,['x','y','z']] - Position.loc[i+1,['x','y','z']] if i!=len(Position)-1 else Position.loc[i-1,['x','y','z']] - Position.loc[i,['x','y','z']] for i in range(len(Position))]) 
    r_dot_a=[r.iloc[i].dot(a.iloc[i]) for i in range(len(r))]
    r['modulo_r'] = r.apply(lambda x:np.linalg.norm(x), axis='columns')
    a['modulo_a'] = a.apply(lambda x:np.linalg.norm(x), axis='columns')

    theta = [np.degrees(np.arccos(r_dot_a[i]/(r.modulo_r[i]*a.modulo_a[i]))) for i in range(len(r_dot_a))]

    a_norm = a[['x','y','z']].apply(lambda x:x/np.linalg.norm(x), axis='columns')

    r1 = calc_point - (Position - a_norm*fuente.length/20)
    r1_dot_a=[r1.iloc[i].dot(a[['x','y','z']].iloc[i]) for i in range(len(r1))]
    r1['modulo_r1'] = r1.apply(lambda x:np.linalg.norm(x), axis='columns')
    theta_1 = np.array([np.degrees(np.arccos(r1_dot_a[i]/(r1.modulo_r1[i]*a.modulo_a[i]))) for i in range(len(r1_dot_a))])

    r2 = calc_point - (Position + a_norm*fuente.length/20)
    r2_dot_a=[r2.iloc[i].dot(a[['x','y','z']].iloc[i]) for i in range(len(r2))]
    r2['modulo_r2'] = r2.apply(lambda x:np.linalg.norm(x), axis='columns')
    theta_2 = np.array([np.degrees(np.arccos(r2_dot_a[i]/(r2.modulo_r2[i]*a.modulo_a[i]))) for i in range(len(r2_dot_a))])

    beta = np.radians(theta_2 - theta_1)

    GL0 = 2*np.arctan(fuente.length/20)/(fuente.length/10)
    GL_r_th = np.array([1/(r.modulo_r[i]**2-(fuente.length/20)**2) if (theta[i]==0 or theta[i]==180) else beta[i]/((fuente.length/10)*r.modulo_r[i]*np.sin(np.radians(theta[i]))) for i in range(len(beta))])

    g_r = np.interp(r.modulo_r*10,fuente.RadialDoseFuntion['r(mm)'],fuente.RadialDoseFuntion['g(r)'])

    x,y = np.meshgrid(np.linspace(0,180,37),np.linspace(0,50,11))
    # Anisotropy2D.drop('r(mm)\\theta(°)',axis=1)
    f = interpolate.interp2d(x,y,np.array(fuente.Anisotropy2D),kind='cubic')

    F_r_th = np.array([(f(theta[i],r.modulo_r[i]*10))[0] for i in range(len(r))])
    
    return Sk*fuente.DoseRateConstant*(GL_r_th/GL0)*g_r*F_r_th.T

DR = Dose_Rate(Position,calc_point,fuente,PlanDate)

SyntaxError: ignored

In [12]:
 from Dose import Dose

In [13]:
Dose([Position],[calc_point],fuente,PlanDate)

/home/jaimeh94/anaconda3/envs/pydicomenv/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=37,17 m=407 fp=0.000002 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


[613.4720060955584]

In [35]:
def Dose(Catheters, Calc_Matrix, fuente, PlanDate):
    """
    This funtion return a matrix of dose in the space
    """
    DoseperMatrix=[]
    for calc_point in Calc_Matrix:
        DoseperCatheter=[]
        for Position in Catheters:
            DoseRate=Dose_Rate(Position[['x','y','z']],calc_point,fuente,PlanDate)
            DoseperDwell = DoseRate*np.array(Position['time']/3600)
            DoseperCatheter.append(DoseperDwell.sum())
        DoseperCatheter = np.array(DoseperCatheter)
        DoseperMatrix.append(DoseperCatheter.sum())
    return DoseperMatrix

In [50]:
from scipy import interpolate
Dose([Position],[calc_point],fuente,PlanDate)

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/_fitpack_impl.py:976: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=37,17 m=407 fp=0.000002 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


[613.4720060955584]

In [49]:
Position

,x,y,z,time
0,-2.187756,0.499022,-1.485154,296.955
1,-2.361418,0.307981,-2.451252,109.876
2,-2.532364,0.118206,-3.417977,44.544
3,-2.644078,-0.043958,-4.398397,109.876
4,-2.755792,-0.206122,-5.378817,296.961
